In [1]:
import aocd
import numpy as np
from collections import Counter
from itertools import chain
data = list([int(d) for d in aocd.get_data(day=23).split(",")])
print(data)

[3, 62, 1001, 62, 11, 10, 109, 2249, 105, 1, 0, 1503, 1097, 1666, 767, 2152, 1740, 1874, 1270, 703, 1775, 1606, 1029, 2043, 1971, 1169, 1637, 1470, 2119, 2181, 600, 1134, 2080, 1841, 1303, 1806, 1379, 1439, 635, 1410, 1940, 827, 924, 1206, 1332, 571, 1544, 860, 2006, 969, 893, 1573, 1909, 1237, 796, 1707, 2218, 672, 1062, 1000, 736, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 64, 1008, 64, -1, 62, 1006, 62, 88, 1006, 61, 170, 1105, 1, 73, 3, 65, 21001, 64, 0, 1, 20101, 0, 66, 2, 21101, 0, 105, 0, 1105, 1, 436, 1201, 1, -1, 64, 1007, 64, 0, 62, 1005, 62, 73, 7, 64, 67, 62, 1006, 62, 73, 1002, 64, 2, 132, 1, 132, 68, 132, 1001, 0, 0, 62, 1001, 132, 1, 140, 8, 0, 65, 63, 2, 63, 62, 62, 1005, 62, 73, 1002, 64, 2, 161, 1, 161, 68, 161, 1101, 1, 0, 0, 1001, 161, 1, 169, 101, 0, 65, 0, 1101, 0, 1, 61, 1101, 0, 0, 63, 7, 63, 67, 62, 1006, 62, 203, 1002, 63, 2, 194, 1, 68, 194, 194, 1006, 0, 73, 1001, 63, 1, 63, 1106, 0, 178, 21101, 0, 210, 0, 105, 1, 69, 2102, 1, 1, 70, 1102, 0, 1, 63, 7, 63, 71, 6

In [2]:
IN_SIZE = {1: 4, 2: 4, 3: 2, 4: 2, 5: 3, 6: 3, 7: 4, 8: 4, 9: 2, 99: 1}
IN_NAME =  {1: "add", 2: "mul", 3: "rd", 4: "prnt", 5: "jnz", 6: "jz", 
            7: "lt", 8: "eq", 9: "bas", 99: "ret"}
EXT_MEM = 1000

class Process():  # wrapper for generator
    def __init__(self, data, ptr=0, dbg=False): 
        self.d = data[:]+[0]*EXT_MEM  # copy + extend memory
        self.done = False
        self.base = 0
        self.ptr = ptr
    
    def parse_ins(self, ptr, dbg=False):
        param = [0, 0, 0]
        ins = self.d[ptr]%100
        modes = [self.d[ptr]//10**e%10 for e in range(2,5)]
        for i, mode in enumerate(modes):
            size = IN_SIZE[ins]-1
            if i < size:
                p = ptr+1+i
                if mode == 0:  param[i] = self.d[p]          # position
                if mode == 1:  param[i] = p                  # intermediate
                if mode == 2:  param[i] = self.base+self.d[p]# relative
        if dbg: print(ptr, IN_NAME[ins], param[:size], 
                      self.d[ptr:ptr+4], sep = "\t")# debug print
        return [ins] + param

    def process(self, inp, dbg=False): 
        out = []; ptr = self.ptr; d = self.d  # initializations
        parse = lambda i: [i%100] + [i//10**e%10 for e in range(2,5)]
        while ptr < len(d):                                     # stop on EOF
            ins, p1, p2, p3 = self.parse_ins(ptr, dbg=dbg)
            # if dbg:print(ptr, d)                              # debug print
            if   ins == 1: d[p3] = d[p1] + d[p2]                # add
            elif ins == 2: d[p3] = d[p1] * d[p2]                # mul
            elif ins == 3:                                      # read
                if not inp: self.ptr = ptr; return out;         # wait/flush
                d[p1] = inp.pop(0)                              # read
            elif ins == 4: out.append(d[p1])                    # print
            elif ins == 5: ptr = d[p2]-3 if     d[p1] else ptr  # jnz
            elif ins == 6: ptr = d[p2]-3 if not d[p1] else ptr  # jz
            elif ins == 7: d[p3] = int(d[p1] < d[p2])           # lt
            elif ins == 8: d[p3] = int(d[p1] == d[p2])          # eq
            elif ins == 9: self.base += d[p1]                   # base
            elif ins ==99: self.done=True; return out           # ret
            else: print(f"invalid opcode {ins} @ {ptr}")        # err
            ptr += IN_SIZE[ins] # jmp is compensated with -3    # move ptr

    def process_str(self, inp):
        out = self.process([ord(c) for c in inp])
        return "".join([chr(i) if i in range(256) else str(i) for i in out])

# tests in other files

In [3]:
proc, packets, nat = [], [], []
c = Counter()

# init processors and send -1 package
for i in range(50):
    proc.append(Process(data))
    packets += proc[i].process([i, -1])
    
# network loop: send and collect packets 
for i in range(100): # repeat arbitrary amount of times
    while packets:   # false if empty
        dest, x, y = packets.pop(0), packets.pop(0), packets.pop(0)
        if dest == 255:       # 255 package goes to nat
            if i == 0: a = y  # save first nat package for later
            nat = [x,y]       # overwrite nat package
        else:                 # normal case: process package
            packets+=proc[dest].process([x,y])
    
    # after all packets have been sent, send nat package to proc0
    packets += proc[0].process(nat[:])
    
    # check if a message was sent twice
    c.update({nat[1]: 1})     # increase counter for current y by 1
    mc = c.most_common()[0]   # tuple (most_common_element, cnt)
    if mc[1] == 2: b = mc[0]; break

aocd.submit(a, day=23)

answer a: 19937
submitting for part b (part a is already completed)
posting 19937 to https://adventofcode.com/2019/day/23/answer (part b) token=...2749


You don't seem to be solving the right level.  Did you already complete it? [Return to Day 23]


<Response [200]>

In [4]:
aocd.submit(b, day=23)

answer a: 19937
submitting for part b (part a is already completed)
posting 13758 to https://adventofcode.com/2019/day/23/answer (part b) token=...2749


You don't seem to be solving the right level.  Did you already complete it? [Return to Day 23]


<Response [200]>